In [1]:
import sys
from pathlib import Path

sys.path.append(Path(".").parent.resolve().as_posix())

import numpy as np
from jesse import helpers, indicators, research

warmup_1m, trading_1m = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "1m",
    helpers.date_to_timestamp("2024-12-01"),
    helpers.date_to_timestamp("2024-12-31"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

In [2]:
from research_tools import TrailingStopLabel

trailing_stop_label = TrailingStopLabel(trading_1m, n_bar=15, k=1.5)
trailing_stop_label.labels.shape

(43200,)

In [3]:
np.unique(trailing_stop_label.labels, return_counts=True)

(array([-1,  0,  1], dtype=int8), array([11106, 21516, 10578]))

In [4]:
np.unique(trailing_stop_label.bar_duration, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16]),
 array([21516,  2441,  3630,  3291,  2653,  2078,  1682,  1346,  1016,
          843,   685,   557,   445,   352,   310,   271,    84]))

In [5]:
real_r = np.abs(trailing_stop_label.realized_r)
real_r = real_r[real_r > 0]
real_r.min()

0.0002526517910906856

In [6]:
real_r.max()

0.035923227998711504

In [8]:
from custom_indicators.td_sequential import td_sequential

# timestamp, open, close, high, low, volume
buy_count, sell_count = td_sequential(
    trading_1m, sequential=True, aggressive=True, stealth_actions=True
)

In [9]:
np.unique(buy_count, return_counts=True)

(array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
        13., 14.]),
 array([11722,  1178,  1106,  1111,  1093,  1125,  1057,   940,   866,
          838,   736,   729,   704,  8455, 11540]))